<a href="https://colab.research.google.com/github/deepkse/QueryOptimizer/blob/main/QueryOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [9]:
from openai import OpenAI
import re

# Provide OpenAI API Key
client = OpenAI(api_key='**********')

def optimize_sql_query(query, dbms):
    # Prompt for LLM
    prompt = f"""
    As an expert SQL query optimizer, your task is to optimize the following SQL query for {dbms} without relying on indexes or query plans.

    Original query:
    {query}

    Please provide an optimized version of this query, focusing on the following aspects:
    1. Simplify complex expressions
    2. Eliminate redundant conditions
    3. Optimize JOIN operations (e.g., consider using EXISTS instead of IN for subqueries)
    4. Use appropriate aggregate functions
    5. Minimize subqueries where possible
    6. Optimize WHERE clauses
    7. Use appropriate DBMS-specific optimizations for {dbms}

    Your response should include:
    1. The optimized SQL query
    2. A detailed explanation of each optimization made
    3. If no optimizations are possible, explain why the original query is already optimal

    Format your response as follows:
    Optimized query:
    [Your optimized SQL query here]

    Explanation:
    [Your detailed explanation here]
    """

    try:
        # Call the OpenAI API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a SQL query optimization assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        # Extract the optimized query and explanation
        ai_response = response.choices[0].message.content
        optimized_query = re.search(r'Optimized query:(.*?)Explanation:', ai_response, re.DOTALL)
        explanation = re.search(r'Explanation:(.*)', ai_response, re.DOTALL)

        if optimized_query and explanation:
            return optimized_query.group(1).strip(), explanation.group(1).strip()
        else:
            return "Error: Could not parse the AI response.", "Please try again with a different query."

    except Exception as e:
        return f"Error: {str(e)}", "An error occurred while processing the query."

while True:
    print("\nSQL Query Optimizer")
    print("-------------------")
    query = input("Enter your SQL query (or 'quit' to exit): ")

    if query.lower() == 'quit':
        break

    dbms = input("Enter the target DBMS (e.g., MySQL, PostgreSQL, Oracle): ")

    optimized_query, explanation = optimize_sql_query(query, dbms)

    print("\nOptimized Query:")
    print(optimized_query)
    print("\nExplanation of optimizations:")
    print(explanation)


SQL Query Optimizer
-------------------
Enter your SQL query (or 'quit' to exit): SELECT customer_name, order_date FROM customers WHERE customer_id IN (     SELECT customer_id     FROM orders     WHERE order_total > 1000 )
Enter the target DBMS (e.g., MySQL, PostgreSQL, Oracle): Oracle

Optimized Query:
```sql
SELECT c.customer_name, o.order_date 
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_total > 1000;
```

Explanation of optimizations:
1. Simplify complex expressions: The original query had a nested subquery using `IN` which can be simplified by using a JOIN operation instead.
   
2. Eliminate redundant conditions: The JOIN condition `c.customer_id = o.customer_id` ensures that only the relevant customer records are fetched based on the common customer_id field.

3. Optimize JOIN operations: By using a JOIN between the `customers` and `orders` tables, we directly retrieve the required data without the need for a separate subquery.

4. Use appropria